## Laser Calibration ```gcode``` examples.

Some code to 'calibrate' laser settings to card stock.

In [22]:
# Imports and autoreload for development
%load_ext autoreload
%autoreload 2
import gcode
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import grbl
cnc = grbl.Grbl(port="/dev/ttyUSB0")

# Line Calibration

Vary feed rate, power and dynamic powering.

In [20]:
cnc.cmd("$100=80.125")
cnc.cmd("$101=80.125")

['ok', 'ok']

In [21]:
def init(**kwargs):
    """
    Basic Machine Init.
    """
    prog = gcode.GCode(**kwargs)
    prog.G21()
    prog.G90()
    prog.G92(X=0, Y=0)
    prog.G1(F=200)
    prog.G0(F=200)
    return prog

In [26]:
# Create an empty GCode program with the above function, some saner defaults than an empty program.
prog = init(machine=cnc)
# Line spacing to use.
line_spacing=2
# Generate points for a 10mm line
line_points = gcode.hline(Xf=10)
for p, power in enumerate(np.arange(25, 255, 25)):
    # Use Absolute coordinates
    prog.G90()
    # Go to a the start of a row, each power has its own row.
    prog.G0(X=0, Y=(p+1)*line_spacing)
    # Relative coordinates for each drawn line.
    prog.G91()
    # Non-dynamic power on cardstock makes holes. 
    for d, dynamic_power in enumerate([True]):
        # For each of the given feed rates
        for f, feed in enumerate(np.arange(30, 360+1, 30)): 
                prog+=gcode.Line(points=line_points, power=power, feed=feed, dynamic_power=dynamic_power)
prog.G90()
prog.G0(X=0, Y=0)

In [30]:
(np.arange(30, 360+1, 30))

array([ 30,  60,  90, 120, 150, 180, 210, 240, 270, 300, 330, 360])

In [31]:
np.arange(25, 255, 25)

array([ 25,  50,  75, 100, 125, 150, 175, 200, 225, 250])

In [33]:
cnc.run(prog)

100%|██████████| 517/517 [13:53<00:00,  1.61s/it] 


849.3127617835999

In [34]:
cnc.status

'<Idle|WPos:0.000,0.000,0.000|Bf:15,127|FS:0,0>'

In [35]:
cnc.cmd("G0X10")

['ok', 'ok']

In [37]:
cnc.status

'<Idle|WPos:9.997,0.000,0.000|Bf:15,127|FS:0,0>'